In [21]:
import json
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer

f1 = open(r'NER_train.json' , 'r' , encoding='utf-8')
f2 = open(r'NER_test.json' , 'r' , encoding='utf-8')
f3 = open(r'NER_val.json' , 'r' , encoding='utf-8')

train_data = list(json.load(f1).values())
test_data = list(json.load(f2).values())
val_data = list(json.load(f3).values())


f1.close()
f2.close()
f3.close()

texts = []

for i in train_data + test_data + val_data:
    texts.append(i['text'])

Tokenizer = Tokenizer(lower=False)
Tokenizer.fit_on_texts(texts)

sequences = Tokenizer.texts_to_sequences(texts)

labels = []
max_sentence_length = 0

for i in train_data + test_data + val_data:
    for j in i['labels']:
        max_sentence_length = max(max_sentence_length , len(i['text'].split(' ')))
        if j not in labels:
            labels.append(j)

labels.append('UNKNNOWN')
labels.sort()

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(labels)

label_encoder.transform(labels)

from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def preprocess_data(data, Tokenizer, LabelEncoder):
    global max_sentence_length
    
    texts = []
    for i in data:
        texts.append(i['text'])

    x = Tokenizer.texts_to_sequences(texts)
    x = pad_sequences(x, maxlen=max_sentence_length, padding='post')


    y = [[label if label in LabelEncoder.classes_ else 'UNKNOWN' for label in item['labels']] for item in data]
    y = [LabelEncoder.transform(sublist).tolist() for sublist in y]
    y = pad_sequences(y, maxlen=max_sentence_length, padding='post')
    y = to_categorical(y, num_classes=len(LabelEncoder.classes_))
    
    return x, y

x_train , y_train = preprocess_data(train_data , Tokenizer , label_encoder)
x_val , y_val = preprocess_data(val_data , Tokenizer , label_encoder)
x_test , y_test = preprocess_data(test_data , Tokenizer , label_encoder)


In [22]:
task1_models_dict = {
    "RNN , Word2Vec" : "t1_model1_word2vec.pkl" , "RNN , GloVe" : "t1_model1_glove.pkl" , "RNN , FastText" : "t1_model1_fasttext.pkl" ,
    "LSTM , Word2Vec" : "t1_model2_word2vec.pkl" , "LSTM , GloVe" : "t1_model2_glove.pkl" , "LSTM , FastText" : "t1_model2_fasttext.pkl" ,
    "GRU , Word2Vec" : "t1_model3_word2vec.pkl" , "GRU , GloVe" : "t1_model3_glove.pkl" , "GRU , FastText" : "t1_model3_fasttext.pkl" , 
    "BiLSTM , Word2Vec" : "t1_model4_word2vec.pkl" , "BiLSTM , GloVe" : "t1_model4_glove.pkl" , "BiLSTM , FastText" : "t1_model4_fasttext.pkl",
    }

In [23]:
from tensorflow.keras.models import load_model
from sklearn.metrics import f1_score

results = []

print("Loading models")
for model_type, model_name in task1_models_dict.items():
    model = load_model(model_name)
    if model_type == "model5 , Word2Vec" or model_type == "model5 , GloVe" or model_type == "model5 , FastText":
        print("\n\n\n")
        y_test_sparse = np.argmax(y_test, axis=-1)
        y_test_pred = model.predict(x_test)
        accuracy = np.mean([np.array_equal(y_test_pred[i], y_test_sparse[i]) for i in range(len(y_test_sparse))])
        f1 = f1_score(y_test_sparse.flatten(), y_test_pred.flatten(), average='macro')
        results.append(f"{model_type} - Test accuracy: {accuracy} - Macro F1: {f1}")

    else:
        accuracy = model.evaluate(x_test, y_test, verbose=0)
        y_pred_test = model.predict(x_test).argmax(axis=-1)
        y_true_test = y_test.argmax(axis=-1)
        f1_test = f1_score(y_true_test.flatten(), y_pred_test.flatten(), average='macro')
        results.append(f"{model_type} - Test accuracy: {accuracy[1]} - Macro F1: {f1_test}")



Loading models
30/30 [==============================] - 1s 17ms/step
RESULTS FOR TASK-2
RNN , Word2Vec - Test accuracy: 0.952263355255127 - Macro F1: 0.49989753746823395
RNN , GloVe - Test accuracy: 0.9718673229217529 - Macro F1: 0.5203911125428325
RNN , FastText - Test accuracy: 0.9722954034805298 - Macro F1: 0.5400026806171008
LSTM , Word2Vec - Test accuracy: 0.9615604281425476 - Macro F1: 0.6111662576194354
LSTM , GloVe - Test accuracy: 0.9741723537445068 - Macro F1: 0.5422426663992475
LSTM , FastText - Test accuracy: 0.9732064604759216 - Macro F1: 0.5356367228482181
GRU , Word2Vec - Test accuracy: 0.9638984203338623 - Macro F1: 0.6450631351969754
GRU , GloVe - Test accuracy: 0.975588321685791 - Macro F1: 0.586641436748892
GRU , FastText - Test accuracy: 0.9741723537445068 - Macro F1: 0.5611531893294935
BiLSTM , Word2Vec - Test accuracy: 0.9808899760246277 - Macro F1: 0.7615969194253047
BiLSTM , GloVe - Test accuracy: 0.9833157658576965 - Macro F1: 0.7413868913900855
BiLSTM , FastTe

In [24]:
print("RESULTS FOR TASK-1")
for result in results:
    print(result)

RESULTS FOR TASK-1
RNN , Word2Vec - Test accuracy: 0.952263355255127 - Macro F1: 0.49989753746823395
RNN , GloVe - Test accuracy: 0.9718673229217529 - Macro F1: 0.5203911125428325
RNN , FastText - Test accuracy: 0.9722954034805298 - Macro F1: 0.5400026806171008
LSTM , Word2Vec - Test accuracy: 0.9615604281425476 - Macro F1: 0.6111662576194354
LSTM , GloVe - Test accuracy: 0.9741723537445068 - Macro F1: 0.5422426663992475
LSTM , FastText - Test accuracy: 0.9732064604759216 - Macro F1: 0.5356367228482181
GRU , Word2Vec - Test accuracy: 0.9638984203338623 - Macro F1: 0.6450631351969754
GRU , GloVe - Test accuracy: 0.975588321685791 - Macro F1: 0.586641436748892
GRU , FastText - Test accuracy: 0.9741723537445068 - Macro F1: 0.5611531893294935
BiLSTM , Word2Vec - Test accuracy: 0.9808899760246277 - Macro F1: 0.7615969194253047
BiLSTM , GloVe - Test accuracy: 0.9833157658576965 - Macro F1: 0.7413868913900855
BiLSTM , FastText - Test accuracy: 0.9844463467597961 - Macro F1: 0.7194714639177093

NAMED ENTITY EXTRACTION

1) NER for each model

In [25]:
test_input = x_test

for model_type, model_name in task1_models_dict.items():
    if model_type not in ["model5 , Word2Vec", "model5 , GloVe", "model5 , FastText"]:
        model = load_model(model_name)
        y_pred_test = model.predict(test_input).argmax(axis=-1)
        y_true_test = y_test.argmax(axis=-1)
        print(f"Model: {model_type} , {model_name}")

        entities_dict = {
            "STATUTE": [],
            "PRECEDENT": [],
            "JUDGE": [],
            "GPE": [],
            "OTHER_PERSON": [],
            "DATE": [],
            "PROVISION": [],
            "CASE_NUMBER": [],
            "COURT": [],
            "ORG": [],
            "RESPONDENT": [],
            "PETITIONER": [],
            "WITNESS": []
        }

        for item in test_data:
            text, labels = item['text'].split(), label_encoder.inverse_transform(y_pred_test[test_data.index(item)])
            entity, type = '', None
            for word, label in zip(text, labels):
                if label.startswith('B'):
                    if entity: entities_dict.get(type, []).append(entity)
                    entity, type = word, label[2:]
                elif label.startswith('I') and type == label[2:]:
                    entity += ' ' + word
                else:
                    if entity: entities_dict.get(type, []).append(entity)
                    entity, type = '', None
            if entity: entities_dict.get(type, []).append(entity)

        print(entities_dict)

30/30 [==============================] - 0s 5ms/step
Model: RNN , Word2Vec , t1_model1_word2vec.pkl
{'STATUTE': ['IPC', 'Income', 'CPC', 'Constitution', 'Constitution', 'Rent Act', 'Indian', 'RPC', 'NDPS Act 1985', 'Constitution', 'IPC', 'Indian Arms', 'IPC', 'Telangana', 'Rent Act', 'Tamil Nadu', 'Commercial', 'Commercial Courts Act', 'Constitution', 'CPC', 'Old Rules to', 'Motor Vehicle Accident Claims', 'Constitution', 'Income Tax Act 1961', 'Constitution', 'Securitisation and Reconstruction of', 'Constitution', 'Indian', 'Constitution', 'GKN', 'Constitution of', 'Civil', 'Cr P C', 'IPC', 'CPC', 'Code', 'Constitution', 'Rent Act', 'Arms Act', 'IPC', 'Code', 'Constitution', 'IPC', 'TADA', 'Motor Accident', 'Amendment Act and', 'Constitutional', 'SAPS', 'Gujarat', 'Cr PC', 'Civil', 'Code', 'Constitution', 'Constitution', 'Constitution', 'Dowry Prohibition Act 19619', 'IPC', 'Constitution', 'Constitution', 'Constitution', 'Cr P C', 'Interest', 'Provisions', 'Indian Patent', 'Cr PC', 'C

2) NER of test set

In [26]:
entities_dict = {
    "STATUTE": [],
    "PRECEDENT": [],
    "JUDGE": [],
    "GPE": [],
    "OTHER_PERSON": [],
    "DATE": [],
    "PROVISION": [],
    "CASE_NUMBER": [],
    "COURT": [],
    "ORG": [],
    "RESPONDENT": [],
    "PETITIONER": [],
    "WITNESS": []
}

for item in test_data:
    text, labels = item['text'].split(), item['labels']
    entity, type = '', None
    for word, label in zip(text, labels):
        if label.startswith('B'):
            if entity: entities_dict.get(type, []).append(entity)
            entity, type = word, label[2:]
        elif label.startswith('I') and type == label[2:]:
            entity += ' ' + word
        else:
            if entity: entities_dict.get(type, []).append(entity)
            entity, type = '', None
    if entity: entities_dict.get(type, []).append(entity)


entities_dict

{'STATUTE': ['Constitution',
  'Hyderabad Land Revenue Act 1317',
  'SG Act',
  'PC Act',
  'IPC',
  'Income tax Act 1961',
  'CPC',
  'Rajasthan Court Fee and Suit valuation Amendment Act 2009',
  'Municipality Act',
  'Constitution of India',
  'IPC',
  'SARFAESI',
  'Constitution of India',
  'Negotiable Instruments Act 26 of 1881',
  'Rent Act',
  'Indian Evidence Act',
  'Finance Bill 2021',
  'PF and ESI Act',
  'PITA Act',
  'ITP Act',
  'RPC',
  'NDPS Act 1985',
  'NDPS Act 1985',
  'IPC',
  'Indian Arms Ac',
  'IPC',
  'Indian Railways Code',
  'DV Act',
  'Rent Act',
  'Punjab Act 2 of 1985',
  'Amendment Act 1988',
  'Amendment Act 1988',
  'Commercial Courts Act',
  'Constitution of India',
  'CPC',
  'Land Acquisition Act 1894',
  'Constitution',
  'SARFAESI Act 2002',
  'Customs Act',
  'FERA',
  'RFCTLARR Act',
  'RFCTLARR Act',
  'Finance Act 2021',
  'Income Tax Act 1961',
  'Arms Act',
  'Constitution of India',
  'Kerala Court Fees and Suits Valuation Act 1959',
  'S